# Research Start

In [ ]:
from datetime import datetime
from scipy import stats
import backtrader as bt
# import yfinance_ez as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker
# import fxcmpy
import os

%matplotlib inline

### Creating Futures Time Series

In [2]:
# Location of the tradingview futures continuous csv files
# csv_loc = 'data/tv_futures'
csv_loc = 'data/tv_futures_daily'

tickers = os.listdir(csv_loc)

In [3]:
"""
Create a dictionary where the key is the ticker
and the value is a pandas dataframe of the OHLC time series
"""
data_tv = {}
for file in tickers:
    ticker = file.split('_')[-1].split()[0][:-3]
    exchange = file.split('_')[0]
    data_tv[ticker] = pd.read_csv(f'{csv_loc}/{file}',
                                        index_col=0,
                                        usecols=[0, 4],
                                        parse_dates=[0],
                                        date_parser=lambda col: pd.to_datetime(col, unit='s'))

# Comment out on Saturdays
for key, item in data_tv.items():
    item.drop(item.tail(1).index, inplace=True)

In [4]:
data_tv.keys()

dict_keys(['HE', 'UB', 'FDAX', '6B', 'NQ', 'GC', 'FGBL', '6A'])

In [5]:
data_tv['NQ'].tail(5)

,close
time,
2020-08-31 22:00:00,12312.50
2020-09-01 22:00:00,12411.50
2020-09-02 22:00:00,11800.50
2020-09-03 22:00:00,11548.75
2020-09-07 22:00:00,11060.50


In [6]:
def volatility(ts, period=24):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(period).std().iloc[-1]

In [7]:
# Create an empty DataFrame to store score
ins_risk_table = pd.DataFrame(columns=['ticker', 'last_date', 'ins_risk'])

# How many (series) candles back for std dev calculation?
vola_window = 24

# Loop the dictionary and calculate the momentum_score, then append it to pandas
for ticker, timeseries in data_tv.items():
    ins_risk = volatility(timeseries['close'], vola_window) * 16
    last_date = timeseries.index[-1]
    ins_risk_table = ins_risk_table.append({'ticker': ticker,
                                            'last_date': last_date,
                                            'ins_risk': ins_risk},
                                           ignore_index=True)

In [8]:
ins_risk_table.sort_values('ticker').to_clipboard()
ins_risk_table.sort_values('ticker')

,ticker,last_date,ins_risk
7,6A,2020-09-07 22:00:00,0.095039
3,6B,2020-09-07 22:00:00,0.115260
2,FDAX,2020-09-08 06:00:00,0.190778
6,FGBL,2020-09-08 06:00:00,0.071606
5,GC,2020-09-07 22:00:00,0.247845
0,HE,2020-09-07 13:30:00,0.414549
4,NQ,2020-09-07 22:00:00,0.290257
1,UB,2020-09-07 22:00:00,0.164789


In [9]:
data_tv['NQ']['close'].iloc[-757:].to_clipboard()
# data_tv['NQ'].tail(1).index
len(data_tv['NQ'])

1031

In [10]:
data_tv['NQ']['close'].iloc[-757:]

time
2017-09-06 22:00:00     5970.00
2017-09-07 22:00:00     5924.50
2017-09-10 22:00:00     5990.75
2017-09-11 22:00:00     6004.50
2017-09-12 22:00:00     6009.00
                         ...   
2020-08-31 22:00:00    12312.50
2020-09-01 22:00:00    12411.50
2020-09-02 22:00:00    11800.50
2020-09-03 22:00:00    11548.75
2020-09-07 22:00:00    11060.50
Name: close, Length: 757, dtype: float64

In [9]:
data_tv['UB']['close'].iloc[-757:].to_clipboard()
len(data_tv['UB'])

1281

In [10]:
data_tv['UB']['close'].tail(5)

time
2020-08-31 22:00:00    222.15625
2020-09-01 22:00:00    224.37500
2020-09-02 22:00:00    225.90625
2020-09-03 22:00:00    220.21875
2020-09-07 22:00:00    222.46875
Name: close, dtype: float64

In [10]:
data_tv['6B']['close'].iloc[-757:].to_clipboard()
len(data_tv['6B'])

1281

In [11]:
data_tv['GC']['close'].iloc[-757:].to_clipboard()
len(data_tv['GC'])

1287

In [16]:
data_tv['FDAX']['close'].iloc[-757:].to_clipboard()
len(data_tv['FDAX'])

1023

In [18]:
data_tv['FGBL']['close'].iloc[-757:].to_clipboard()
len(data_tv['FGBL'])

1023

In [14]:
data_tv['6A']['close'].iloc[-757:].to_clipboard()
len(data_tv['6A'])

1281

In [15]:
data_tv['HE']['close'].iloc[-757:].to_clipboard()
len(data_tv['HE'])

1281

In [16]:
for key, item in data_tv.items():
    print(item.index[-1])

2020-09-03 22:00:00
2020-09-04 06:00:00
2020-09-04 01:00:00
2020-09-04 00:00:00
2020-09-03 21:00:00
2020-09-03 22:00:00
2020-09-03 22:00:00
2020-09-03 22:00:00
2020-09-04 00:00:00
2020-09-03 22:00:00
2020-09-03 22:00:00
2020-09-04 08:45:00
2020-09-03 22:00:00
2020-09-04 06:00:00
2020-09-04 00:00:00
2020-09-03 22:00:00
2020-09-03 22:00:00
2020-09-04 06:00:00
2020-09-03 22:00:00
2020-09-04 06:00:00
2020-09-04 08:15:00
2020-09-03 22:00:00
2020-09-03 22:00:00
2020-09-04 00:00:00
2020-09-04 00:00:00
2020-09-04 13:30:00
2020-09-04 00:00:00
2020-09-03 22:00:00
2020-09-04 00:00:00
2020-09-04 06:00:00
2020-09-04 07:30:00
2020-09-03 22:00:00
2020-09-04 06:00:00
2020-09-04 06:00:00
2020-09-03 22:00:00
2020-09-04 00:00:00
2020-09-03 22:00:00
2020-09-03 22:00:00
2020-09-04 12:00:00
2020-09-03 21:46:00
2020-09-03 22:00:00
2020-09-03 22:00:00
2020-09-04 13:30:00
2020-09-03 22:00:00
2020-09-03 22:00:00
2020-09-03 22:00:00
2020-09-03 22:00:00
2020-09-03 22:00:00
2020-09-04 00:00:00
2020-09-03 22:00:00


In [17]:
data_tv['NQ']['ma_2'] = data_tv['NQ']['close'].rolling(2).mean()
data_tv['NQ']['ma_4'] = data_tv['NQ']['close'].rolling(4).mean()
data_tv['NQ']['ma_8'] = data_tv['NQ']['close'].rolling(8).mean()
data_tv['NQ']['ma_16'] = data_tv['NQ']['close'].rolling(16).mean()
data_tv['NQ']['ma_32'] = data_tv['NQ']['close'].rolling(32).mean()
data_tv['NQ']['ma_64'] = data_tv['NQ']['close'].rolling(64).mean()
data_tv['NQ']['ma_128'] = data_tv['NQ']['close'].rolling(128).mean()
data_tv['NQ']['ma_256'] = data_tv['NQ']['close'].rolling(256).mean()

data_tv['NQ']['ma_2_8'] = data_tv['NQ']['ma_2'] - data_tv['NQ']['ma_8']
data_tv['NQ']['ma_4_16'] = data_tv['NQ']['ma_4'] - data_tv['NQ']['ma_16']
data_tv['NQ']['ma_8_32'] = data_tv['NQ']['ma_8'] - data_tv['NQ']['ma_32']
data_tv['NQ']['ma_16_64'] = data_tv['NQ']['ma_16'] - data_tv['NQ']['ma_64']
data_tv['NQ']['ma_32_128'] = data_tv['NQ']['ma_32'] - data_tv['NQ']['ma_128']
data_tv['NQ']['ma_64_256'] = data_tv['NQ']['ma_64'] - data_tv['NQ']['ma_256']
data_tv['NQ']['ins_risk'] = data_tv['NQ']['close'].pct_change().rolling(24).std() * 16

data_tv['NQ']['ma_2_8_risk'] = data_tv['NQ']['ma_2_8'] / data_tv['NQ']['ins_risk']
data_tv['NQ']['ma_4_16_risk'] = data_tv['NQ']['ma_4_16'] / data_tv['NQ']['ins_risk']
data_tv['NQ']['ma_8_32_risk'] = data_tv['NQ']['ma_8_32'] / data_tv['NQ']['ins_risk']
data_tv['NQ']['ma_16_64_risk'] = data_tv['NQ']['ma_16_64'] / data_tv['NQ']['ins_risk']
data_tv['NQ']['ma_32_128_risk'] = data_tv['NQ']['ma_32_128'] / data_tv['NQ']['ins_risk']
data_tv['NQ']['ma_64_256_risk'] = data_tv['NQ']['ma_64_256'] / data_tv['NQ']['ins_risk']

data_tv['NQ']['ma_2_8_scale'] = 180.8
data_tv['NQ']['ma_4_16_scale'] = 124.32
data_tv['NQ']['ma_8_32_scale'] = 83.84
data_tv['NQ']['ma_16_64_scale'] = 57.12
data_tv['NQ']['ma_32_128_scale'] = 38.24
data_tv['NQ']['ma_64_256_scale'] = 25.28

# data_tv['NQ']['ma_2_8_forecast'] = min(20, max(data_tv['NQ']['ma_2_8_risk']*data_tv['NQ']['ma_2_8_scale'], -20))
# data_tv['NQ']['ma_4_16_forecast'] = min(20, max(data_tv['NQ']['ma_4_16_risk']*data_tv['NQ']['ma_4_16_scale'], -20))
# data_tv['NQ']['ma_8_32_forecast'] = min(20, max(data_tv['NQ']['ma_8_32_risk']*data_tv['NQ']['ma_8_32_scale'], -20))
# data_tv['NQ']['ma_16_64_forecast'] = min(20, max(data_tv['NQ']['ma_16_64_risk']*data_tv['NQ']['ma_16_64_scale'], -20))
# data_tv['NQ']['ma_32_128_forecast'] = min(20, max(data_tv['NQ']['ma_32_128_risk']*data_tv['NQ']['ma_32_128_scale'], -20))
# data_tv['NQ']['ma_64_256_forecast'] = min(20, max(data_tv['NQ']['ma_64_256_risk']*data_tv['NQ']['ma_64_256_scale'], -20))


data_tv['NQ'].tail(5).to_clipboard()
data_tv['NQ'].tail(5)

,close,ma_2,ma_4,ma_8,ma_16,ma_32,ma_64,ma_128,ma_256,ma_2_8,...,ma_8_32_risk,ma_16_64_risk,ma_32_128_risk,ma_64_256_risk,ma_2_8_scale,ma_4_16_scale,ma_8_32_scale,ma_16_64_scale,ma_32_128_scale,ma_64_256_scale
time,,,,,,,,,,,,,,,,,,,,,
2020-08-30 22:00:00,12114.00,12052.875,12012.4375,11806.43750,11491.56250,11165.031250,10651.398438,9561.939453,8999.901367,246.43750,...,3985.924361,5221.075416,9962.177084,10262.984503,180.8,124.32,83.84,57.12,38.24,25.28
2020-08-31 22:00:00,12312.50,12213.250,12092.7500,11910.84375,11569.09375,11217.843750,10693.035156,9589.449219,9018.455078,302.40625,...,4249.471513,5371.985625,9985.304723,10268.514197,180.8,124.32,83.84,57.12,38.24,25.28
2020-09-01 22:00:00,12411.50,12362.000,12207.4375,12017.03125,11664.90625,11263.484375,10735.640625,9619.363281,9037.252930,344.96875,...,4643.523733,5726.341821,10131.440488,10465.843377,180.8,124.32,83.84,57.12,38.24,25.28
2020-09-02 22:00:00,11800.50,12106.000,12159.6250,12037.56250,11707.06250,11293.156250,10769.613281,9642.050781,9053.223633,68.43750,...,3078.555833,3876.901571,6828.288144,7098.276463,180.8,124.32,83.84,57.12,38.24,25.28
2020-09-03 22:00:00,11548.75,11674.625,12018.3125,12015.37500,11730.40625,11316.421875,10796.804688,9664.707031,9068.293945,-340.75000,...,2774.486551,3705.920878,6556.463454,6861.303907,180.8,124.32,83.84,57.12,38.24,25.28


In [18]:
(data_tv['NQ']['close'] - (data_tv['NQ']['close'].rolling(10).max() + data_tv['NQ']['close'].rolling(10).min()) / 2) / ((data_tv['NQ']['close'].rolling(10).max() - data_tv['NQ']['close'].rolling(10).min()))

time
2015-07-29 22:00:00         NaN
2015-07-30 22:00:00         NaN
2015-08-02 22:00:00         NaN
2015-08-03 22:00:00         NaN
2015-08-04 22:00:00         NaN
                         ...   
2020-08-30 22:00:00    0.500000
2020-08-31 22:00:00    0.500000
2020-09-01 22:00:00    0.500000
2020-09-02 22:00:00   -0.219247
2020-09-03 22:00:00   -0.500000
Name: close, Length: 1287, dtype: float64